In [1]:
import numpy as np
from scipy import sparse
from scipy.linalg import solve
import scipy.ndimage
from collections import defaultdict
import time
from copy import deepcopy

from scipy.sparse.csgraph import depth_first_tree
from scipy.sparse.csgraph import connected_components
from scipy.sparse.linalg import svds

from scipy.sparse import csc_matrix
from scipy.stats import ortho_group

from scipy.sparse.linalg import cgs

import warnings

In [31]:
class LLF(object):

    def __init__(self, max_iter=100, alpha=1.0, eta=1.0, gamma_x=1.0, gamma_y=1.0, tol=1e-2, n_neighbors=3):
        self._max_iter = max_iter
        self._tol = tol
        self._n_neighbors = n_neighbors
        self._gamma_x, self._gamma_y, self._alpha, self._eta = gamma_x, gamma_y, alpha, eta
        
        self._history = defaultdict(list)
    
    
    def _generate_weight_graph(self, mat):
        data = None
        if mat == 'X':
            data = self._data.toarray()
        elif mat == 'Y':
            data = self._data.toarray().T

        row_pairwise_distances = np.empty((data.shape[0], data.shape[0]))
        is_nan = np.isnan(data)
        
        # iterate over all possible pairs of rows
        for i in range(data.shape[0]):
            # take indices of not nan elements in the first row
            not_nan_indices_i = np.argwhere(is_nan[i] != True).reshape(-1)

            for j in range(i, data.shape[0]):
                # take indices of not nan elements in the second row and then consider intersect of indices 
                # of not nan elements of the first and second rows
                not_nan_indices_j = np.argwhere(is_nan[j] != True).reshape(-1)
                intersect = np.intersect1d(not_nan_indices_i, not_nan_indices_j)
                
                # if intersection is not empty we can compute MSE over observed elemnts in both rows.
                # if i==j then we compute distance between objects and itself, so the distance is equal to zero, thus
                # we assign it to infinity.
                if intersect.shape[0] > 0 and i != j:
                    mse_of_observed_elements = np.sum((data[i][intersect] - data[j][intersect]) ** 2) 
                    mse_of_observed_elements = (mse_of_observed_elements / intersect.shape[0]) ** 0.5
                    
                    #we will bound minima distance between rows for more stable computations, because later
                    #matrix of distances will be inversed element-wise.
                    if mse_of_observed_elements > 1e-2:
                        row_pairwise_distances[i][j] = np.round(mse_of_observed_elements, 2)
                    else:
                        row_pairwise_distances[i][j] = 1e-2
                else:
                    row_pairwise_distances[i][j] = np.inf
        
        # we computed upper triangular part of matrix of pairwise row distances, 
        # now we create full symmetric matrix of them
        il = np.tril_indices(row_pairwise_distances.shape[0], -1)
        row_pairwise_distances[il] = row_pairwise_distances[il[::-1]]
        print('row pairwise {}'.format(mat))
        print(row_pairwise_distances)
        print('\\\\\\')
        
        #for each row we find k-nearest rows, their weights will not be nullified. 
        #All other rows should not be taken into account, so we create mask, where 1 stay on that positions
        #which belong to k-neighbors set and 0 elsewhere.
        ind_of_nearest = (np.repeat(np.arange(row_pairwise_distances.shape[0]).reshape(-1,1), 
                                    self._n_neighbors, axis=1), 
                          np.argsort(row_pairwise_distances, axis=1)[:, :self._n_neighbors])
        mask = np.zeros(row_pairwise_distances.shape)
        mask[ind_of_nearest] = 1

        # Now we remove all cycles in graph.
        # first of all we find all connectrd components in graph
        n_components, component_label = connected_components(mask, directed=False)
        unique_labels, unique_indices = np.unique(component_label, return_index=True)
        result_mask = np.zeros(mask.shape)
        # iterating over connected components we find spanned tree, which containes component
        # and create new mask, such that it does not has cycles.
        for i in range(n_components):
            temporary_mask = depth_first_tree(mask, unique_indices[i], directed=False)
            result_mask = result_mask + temporary_mask
        # now we can compute weight matrix
        weight_matrix = result_mask / row_pairwise_distances
        weight_matrix = 0.5 * (weight_matrix + weight_matrix.T)
        return weight_matrix
    
    def _generate_matrix_of_given_rank_and_shape(self, shape):
        mat = np.zeros(shape)
        np.fill_diagonal(mat, np.random.random(min(shape[0], shape[1])) * max(shape[0], shape[1]))
        print(np.linalg.matrix_rank(mat))
        return mat
        
    def _generate_Ex_matrix(self):
        shape = (self._data.shape[0], self._eps_w.shape[0])
        mat = np.zeros(shape)
        plus_ones_args = (self._eps_w[:, 0], np.arange(self._eps_w.shape[0]))
        min_ones_args = (self._eps_w[:, 1], np.arange(self._eps_w.shape[0]))
        mat[plus_ones_args] = 1
        mat[min_ones_args] = -1
        self._Ex = mat
        
    def _generate_Ey_matrix(self):
        shape = (self._data.shape[1], self._eps_u.shape[0])
        mat = np.zeros(shape)
        plus_ones_args = (self._eps_u[:, 0], np.arange(self._eps_u.shape[0]))
        min_ones_args = (self._eps_u[:, 1], np.arange(self._eps_u.shape[0]))
        mat[plus_ones_args] = 1
        mat[min_ones_args] = -1
        self._Ey = mat
    
    def _generate_initial_matrices(self):
        #self._X_input, self._dim = X, d
        print('generate first mat')
        self._weight_X = self._generate_weight_graph('X')
        print('X weights\n', self._weight_X)
        print('<<<<<<<')
        print('generate second mat')
        self._weight_Y = self._generate_weight_graph('Y')
        print('Y weights\n', self._weight_Y)
        print('<<<<<<<')
        
        # random initialization of training parameters.
        self._X_fac = self._generate_matrix_of_given_rank_and_shape((self._data.shape[0], self._rank))
        self._Y_fac = self._generate_matrix_of_given_rank_and_shape((self._data.shape[1], self._rank))
        print('Yshape1: {}'.format(self._Y_fac.shape))
        
        self._eps_w = np.argwhere(self._weight_X > 0)
        self._eps_w = self._eps_w[self._eps_w[:,0] < self._eps_w[:,1]]
        self._eps_u = np.argwhere(self._weight_Y > 0)
        self._eps_u = self._eps_u[self._eps_u[:,0] < self._eps_u[:,1]]
        
        self._Lambda_constr = np.random.rand(self._rank, self._eps_w.shape[0])
        self._V_constr = np.random.rand(self._rank, self._eps_u.shape[0])
        
        self._generate_Ex_matrix()
        self._generate_Ey_matrix()
        
        self._P = np.zeros((self._rank, self._eps_w.shape[0]))
        self._Q = np.zeros((self._rank, self._eps_u.shape[0]))
        
        
#-------------functions to make updates
    
    def _gen_block_diag_Gx_matrix_with_n_blocks(self):
        """
        This function generates block-diagonal matric for plugging into conjugate gradient method
        """
     
        #we compute the first block independently outside the loop
        #first of all find arguments of observed elements in the first row of data
        args = np.argwhere(~np.isnan(self._data[0].toarray()))[:, 1]
        #take rows of Y_fac matrix which correspond to columns of observed elements
        rows_in_Y = self._Y_fac[args, :]
        res = np.einsum('ij, ik->jk', rows_in_Y, rows_in_Y)
        #inside the loop we repeat the same operations and stack the results along the diagonal
        for i in range(1, self._data.shape[0]):
            args = np.argwhere(~np.isnan(self._data[i].toarray()))[:, 1]
            rows_in_Y = self._Y_fac[args, :]
            res = scipy.linalg.block_diag(res, np.einsum('ij, ik->jk', rows_in_Y, rows_in_Y)) 
        return res
    
    def _gen_matrix_cg_X(self):
        """
        This function computes matrix of system which pushes gradient to zero.
        """
        #initialize result variable with matrix obtained from function 
        result = self._gen_block_diag_Gx_matrix_with_n_blocks()
        #add second term
        result = result + scipy.linalg.kron((self._eta * self._Ex.dot(self._Ex.T) + (self._alpha + 1) * np.eye(self._Ex.shape[0])), np.eye(self._rank))
        return result
            
    def _gen_vector_b_y(self):
        """
        This function creates vector from smaller vectors. The result one will be used to obtain the right part
        of system of linear equations which pushes gradient to zero.
        """
        b_y = None
        for i in range(self._data.shape[0]):
            if i == 0:
                #on the first iteration we initialize vector
                #we find arguments of not nan elements in the first row of data matrix
                args_of_not_nan = np.argwhere(~np.isnan(self._data[0].toarray()))[:, 1]
                #we find not nan elements in 1 row of data matrix and multiply the corresponding rows
                #of Y_fac matrix with them, after that we sum these row and obtain b_y
                b_y = self._Y_fac[args_of_not_nan]
                b_y = b_y * self._data[0].toarray()[0][args_of_not_nan].reshape(-1,1)
                b_y = np.sum(b_y, axis=0)
            else:
                args_of_not_nan = np.argwhere(~np.isnan(self._data[i].toarray()))[:, 1]
                add = np.sum(self._Y_fac[args_of_not_nan] * self._data[i].toarray()[0][args_of_not_nan].reshape(-1,1), 
                             axis=0)
                b_y = np.concatenate((b_y, add))

        flag = np.sum(np.isnan(b_y).astype(int))
        if flag != 0:
            raise Exception('Nans in vector')
    
        return b_y
    
    def _gen_vector_C_y(self):
        """
        This function computes vector from the right part of system of linear equations.
        """
        b_y = self._gen_vector_b_y()
        C = b_y + (self._X_fac.T + self._eta * self._P.dot(self._Ex.T) + self._Lambda_constr.dot(self._Ex.T)).ravel(order='F')
        return C
    
    def _f(self, M):
        vec = M.ravel(order='F')
        mat = self._gen_block_diag_Gx_matrix_with_n_blocks()
        C = self._gen_vector_C_y()
        return 0.5 * vec.reshape(1,-1).dot(mat.dot(vec.reshape(-1,1))) - C.dot(vec)
    
    def _F(self, x):
        res = 0
        for ind in self._not_nan_indices:
            res += (self._data[ind[0], ind[1]] - x[ind[0]].dot(self._Y_fac[ind[1]])) ** 2
        res += self._eta / 2.0 * (np.linalg.norm(self._P - x.T.dot(self._Ex)) ** 2)
        res += np.sum(np.diag(self._Lambda_constr.T.dot(self._P - x.T.dot(self._Ex))))
        res += self._alpha / 2.0 * np.power(np.linalg.norm(x), 2)
        res += 0.5 * np.power(np.linalg.norm(x - self._X_fac), 2)
        return res
    
#-------------------------------------------------------------------     
    
    def _gen_block_diag_Gy_matrix_with_m_blocks(self):
        """
        This function generates block-diagonal matric for plugging into conjugate gradient method
        """
        res = np.zeros((self._rank, self._rank))
        args = np.argwhere(~np.isnan(self._data.T[0].toarray()))[:, 1]
        rows_in_X = self._X_fac[args, :]
        res = np.einsum('ij, ik->jk', rows_in_X, rows_in_X)
        for i in range(1, self._data.shape[1]):
            args = np.argwhere(~np.isnan(self._data.T[i].toarray()))[:, 1]
            rows_in_X = self._X_fac[args, :]
            res = scipy.linalg.block_diag(res, np.einsum('ij, ik->jk', rows_in_X, rows_in_X)) 
        return res
    
    def _gen_matrix_cg_Y(self):
        result = self._gen_block_diag_Gy_matrix_with_m_blocks() 
        result = result + scipy.linalg.kron((self._eta * self._Ey.dot(self._Ey.T) + (self._alpha + 1) * np.eye(self._Ey.shape[0])), 
                                            np.eye(self._rank))
        return result
            
    def _gen_matrix_b_x(self):
        b_x = None
        for i in range(self._data.shape[1]):
            if i == 0:
                args_of_not_nan = np.argwhere(~np.isnan(self._data.T[0].toarray()))[:, 1]
                b_x = np.sum(self._X_fac[args_of_not_nan] * self._data.T[0].toarray()[0][args_of_not_nan].reshape(-1,1),
                             axis=0)
            else:
                args_of_not_nan = np.argwhere(~np.isnan(self._data.T[i].toarray()))[:, 1]
                add = np.sum(self._X_fac[args_of_not_nan] * self._data.T[i].toarray()[0][args_of_not_nan].reshape(-1,1), 
                             axis=0)
                b_x = np.concatenate((b_x, add))
        flag = np.sum(np.isnan(b_x).astype(int))
        if flag != 0:
            raise Exception('Nans in matrix')
        return b_x
    
    def _gen_matrix_C_x(self):
        b_x = self._gen_matrix_b_x()
        C = b_x + (self._Y_fac.T + self._eta * self._Q.dot(self._Ey.T) + self._V_constr.dot(self._Ey.T)).ravel()
        return C
    
#-------------------------------------------------------------------    

    def fit(self, X, d):
        self._not_nan_indices = np.argwhere(~np.isnan(X.toarray()))
        if d > min(X.shape[0], X.shape[1]):
            raise Exception('Rank of X and Y must be less than mininimal dimension of matrix. Got rank {}, minimal dimension {}'.format(d, min(X.shape[0], X.shape[1])))
        self._rank = d
        self._data = X
        self._generate_initial_matrices()
        print('X^T.dot(E_x)')
        print(self._X_fac.T.dot(self._Ex))
        print('+++++++')
        print('Y^T.dot(E_y)')
        print(self._Y_fac.T.dot(self._Ey))
        print('------')
        
        print(self._f(np.ones(self._X_fac.shape)))
        print(self._F(np.ones(self._X_fac.shape)))
        """for it in range(self._max_iter):
            print('ITITITITITITITITITI iteration {}'.format(it))
            for i in range(self._eps_w.shape[0]):
                weight_index = self._eps_w[i]
                #print(self._P[:,i].shape, self._X_fac.T.shape, self._Ex[:, i].shape, self._Lambda_constr[:,i].shape, self._weight_X[weight_index[0], weight_index[1]])
                aux = self._X_fac.T.dot(self._Ex[:, i]) - self._eta ** (-1) * self._Lambda_constr[:,i]
                self._P[:,i] = (aux ) * np.power((1 + self._weight_X[weight_index[0], weight_index[1]] / self._eta), -1)
                
            for i in range(self._eps_u.shape[0]):
                weight_index = self._eps_u[i]
                aux = self._Y_fac.T.dot(self._Ey[:, i]) - self._eta ** (-1) * self._V_constr[:,i]
                self._Q[:,i] = aux * np.power((1 + self._weight_Y[weight_index[0], weight_index[1]] / self._eta), -1)
                
            self._X_fac = cgs(self._gen_matrix_cg_X(), self._gen_matrix_C_y())[0].reshape(self._X_fac.T.shape).T
            self._Y_fac = cgs(self._gen_matrix_cg_Y(), self._gen_matrix_C_x())[0].reshape(self._Y_fac.T.shape).T
            
            self._Lambda_constr = self._Lambda_constr + self._eta * (self._P - self._X_fac.T.dot(self._Ex))
            self._V_constr = self._V_constr + self._eta * (self._Q - self._Y_fac.T.dot(self._Ey))
            
            difference = (self._data - self._X_fac.dot(self._Y_fac.T))[self._not_nan_indices[:, 0], self._not_nan_indices[:, 1]]
            self._history['error'].append(np.mean(np.power(difference, 2)))"""
        #self._gen_matrix_cg_X()
        #self._gen_matrix_C_y()
        #print('-----1111111', self._gen_matrix_C_y())
        #print(np.linalg.norm(self._gen_matrix_cg_X().dot(cgs(self._gen_matrix_cg_X(), self._gen_matrix_C_y())[0]) - self._gen_matrix_C_y()))
        #self._gen_matrix_cg_Y()
        #self._gen_matrix_C_x()
        
            
            
    def solution(self):
        return None
    
    def get_weights_X(self):
        return self._weight_X
    
    def get_weights_Y(self):
        return self._weight_Y
    
    def get_history(self):
        return self._history

In [32]:
a.toarray()

array([[nan,  0., nan,  3., nan],
       [ 0., nan,  1., nan,  2.],
       [nan, nan,  3.,  4., nan],
       [ 1.,  2., nan,  3.,  1.],
       [nan, nan,  1., nan, nan],
       [ 1.,  1., nan,  0.,  1.],
       [ 1., nan,  3.,  1., nan],
       [nan, nan,  2.,  1.,  0.]])

In [33]:
a = sparse.csr_matrix(np.array([[np.nan, 0, np.nan, 3, np.nan],
       [0, np.nan, 1, np.nan, 2],
       [np.nan, np.nan, 3, 4, np.nan],
       [1, 2, np.nan, 3, 1],
       [np.nan, np.nan, 1, np.nan, np.nan],
       [1, 1, np.nan, 0, 1],
       [1, np.nan, 3, 1, np.nan],
       [np.nan, np.nan, 2, 1, 0]]))
llf = LLF()
llf.fit(a, 3)

generate first mat
row pairwise X
[[ inf  inf 1.   1.41  inf 2.24 2.   2.  ]
 [ inf  inf 2.   1.   0.01 1.   1.58 1.58]
 [1.   2.    inf 1.   2.   4.   2.12 2.24]
 [1.41 1.   1.    inf  inf 1.58 1.41 1.58]
 [ inf 0.01 2.    inf  inf  inf 2.   1.  ]
 [2.24 1.   4.   1.58  inf  inf 0.71 1.  ]
 [2.   1.58 2.12 1.41 2.   0.71  inf 0.71]
 [2.   1.58 2.24 1.58 1.   1.   0.71  inf]]
\\\
X weights
 [[0.         0.         0.5        0.         0.         0.
  0.         0.        ]
 [0.         0.         0.25       0.5        0.         0.
  0.         0.        ]
 [0.5        0.25       0.         0.         0.         0.
  0.         0.        ]
 [0.         0.5        0.         0.         0.         0.
  0.35460993 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.         0.
  0.70422535 0.5       ]
 [0.         0.         0.         0.35460993 0.         0.70422535
  0.         0.        ]
 [0

In [14]:
llf.get_history()['error']

[]

In [236]:
a = sparse.csr_matrix(np.array([[np.nan, 0, np.nan, 3, np.nan],
       [0, np.nan, 1, np.nan, 2],
       [np.nan, np.nan, 3, 4, np.nan],
       [1, 2, np.nan, 3, 1],
       [np.nan, np.nan, 1, np.nan, np.nan],
       [1, 1, np.nan, 0, 1],
       [1, np.nan, 3, 1, np.nan],
       [np.nan, np.nan, 2, 1, 0]]))

In [42]:
Y = np.random.rand(a.shape[1], 3)
res = np.zeros((3, 3))
args = np.argwhere(~np.isnan(a[0].toarray())).reshape(-1,)
for arg in args:
    res = res + Y[arg].reshape(-1,1).dot(Y[arg].reshape(1,-1))

for i in range(1, a.shape[0]):
        args = np.argwhere(~np.isnan(a[i].toarray()))[:, 1]
        #print(args)
        elements = Y[args, :]
        #print(elements)
        temp_res = np.zeros((3, 3))
        for arg in args:
            temp_res = temp_res + Y[arg].reshape(-1,1).dot(Y[arg].reshape(1,-1))
        res = scipy.linalg.block_diag(res, temp_res)
            
print('Yshape: {}'.format(res.shape))

Yshape: (24, 24)


In [105]:
Y = np.random.rand(a.shape[1], 3)
res = np.zeros((3, 3))
args = np.argwhere(~np.isnan(a[0].toarray()))[:, 1]
print(args)
for arg in args:
    res = res + Y[arg].reshape(-1,1).dot(Y[arg].reshape(1,-1))

for i in range(1, a.shape[0]):
        args = np.argwhere(~np.isnan(a[i].toarray()))[:, 1]
        for arg in args:
            temp_res = temp_res + Y[arg].reshape(-1,1).dot(Y[arg].reshape(1,-1))
        res = scipy.linalg.block_diag(res, np.einsum('ij, ik->jk', elements, elements))
res.shape

[1 3]


(24, 24)

In [150]:
llf.get_weights_X()[0,0]

0.0

In [99]:
a.toarray()

array([[nan,  0., nan,  3., nan],
       [ 0., nan,  1., nan,  2.],
       [nan, nan,  3.,  4., nan],
       [ 1.,  2., nan,  3.,  1.],
       [nan, nan,  1., nan, nan],
       [ 1.,  1., nan,  0.,  1.],
       [ 1., nan,  3.,  1., nan],
       [nan, nan,  2.,  1.,  0.]])

In [100]:
a.indices

array([0, 2, 3, 4, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
       4, 0, 1, 2, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3], dtype=int32)

In [110]:
a[np.argwhere(~np.isnan(a.toarray()))[:,0], np.argwhere(~np.isnan(a.toarray()))[:,1]]

matrix([[0., 3., 0., 1., 2., 3., 4., 1., 2., 3., 1., 1., 1., 1., 0., 1.,
         1., 3., 1., 2., 1., 0.]])

In [108]:
np.argwhere(~np.isnan(a.toarray()))

array([[0, 1],
       [0, 3],
       [1, 0],
       [1, 2],
       [1, 4],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 3],
       [3, 4],
       [4, 2],
       [5, 0],
       [5, 1],
       [5, 3],
       [5, 4],
       [6, 0],
       [6, 2],
       [6, 3],
       [7, 2],
       [7, 3],
       [7, 4]])

In [111]:
np.concatenate((np.arange(3), np.arange(4)))

array([0, 1, 2, 0, 1, 2, 3])

In [178]:
a = np.random.randint(0,3, (4,5))
a

array([[0, 2, 1, 2, 2],
       [2, 1, 1, 1, 0],
       [1, 2, 2, 1, 0],
       [2, 2, 1, 2, 1]])

In [223]:
np.ravel?

In [ ]:
    def _gen_Gx_matrix(self):
        res = np.zeros((self._rank, self._rank))
        args = np.argwhere(~np.isnan(self._data.T[0].toarray()))[:, 1]
        elements = self._X_fac[args, :]
        res = np.einsum('ij, ik->jk', elements, elements)
        for i in range(1, self._data.shape[1]):
            args = np.argwhere(~np.isnan(self._data.T[i].toarray()))[:, 1]
            elements = self._X_fac[args, :]
            res = scipy.linalg.block_diag(res, np.einsum('ij, ik->jk', elements, elements)) 
        return res
    
    def _gen_matrix_cg_Y(self):
        result = self._gen_Gx_matrix() + scipy.linalg.kron((self._eta * self._Ey.dot(self._Ey.T) + (self._alpha + 1) * np.eye(self._Ey.shape[0])), 
                                                           np.eye(self._rank))
        return result
            
    def _gen_matrix_b_y(self):
        b_y = None
        for i in range(self._data.shape[1]):
            if i == 0:
                args_of_not_nan = np.argwhere(~np.isnan(self._data.T[0].toarray()))[:, 1]
                b_y = np.sum(self._X_fac[args_of_not_nan] * self._data.T[0].toarray()[0][args_of_not_nan].reshape(-1,1),
                             axis=0)
            else:
                args_of_not_nan = np.argwhere(~np.isnan(self._data.T[i].toarray()))[:, 1]
                add = np.sum(self._X_fac[args_of_not_nan] * self._data.T[i].toarray()[0][args_of_not_nan].reshape(-1,1), 
                             axis=0)
                b_x = np.concatenate((b_x, add))
        flag = np.sum(np.isnan(b_x).astype(int))
        if flag != 0:
            raise Exception('Nans in matrix')
        return b_x
    
    def _gen_matrix_C_x(self):
        b_x = self._gen_matrix_b_x()
        C = b_x + (self._Y_fac.T + self._eta * self._Q.dot(self._Ex.T) + self._V_constr.dot(self._Ey.T)).ravel()
        return C

In [220]:
np.arange(12).reshape(3,4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [226]:
np.arange(12).reshape(3,4).ravel(order='F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

In [29]:
rows_in_X = np.random.rand(18).reshape(3,6)
res = np.einsum('ij, ik->jk', rows_in_X, rows_in_X)
res

array([[2.28527561, 1.6442932 , 1.27190273, 1.12878651, 2.1809423 ,
        1.14582379],
       [1.6442932 , 1.33906341, 0.73981177, 0.9920526 , 1.62059768,
        0.7214189 ],
       [1.27190273, 0.73981177, 0.96507636, 0.38741789, 1.17528503,
        0.7861035 ],
       [1.12878651, 0.9920526 , 0.38741789, 0.78981108, 1.12415308,
        0.42622525],
       [2.1809423 , 1.62059768, 1.17528503, 1.12415308, 2.10443688,
        1.06999147],
       [1.14582379, 0.7214189 , 0.7861035 , 0.42622525, 1.06999147,
        0.66021036]])

In [30]:
res = np.zeros((6,6))
for row in rows_in_X:
    res = res + row.reshape(-1,1).dot(row.reshape(1,-1))
res

array([[2.28527561, 1.6442932 , 1.27190273, 1.12878651, 2.1809423 ,
        1.14582379],
       [1.6442932 , 1.33906341, 0.73981177, 0.9920526 , 1.62059768,
        0.7214189 ],
       [1.27190273, 0.73981177, 0.96507636, 0.38741789, 1.17528503,
        0.7861035 ],
       [1.12878651, 0.9920526 , 0.38741789, 0.78981108, 1.12415308,
        0.42622525],
       [2.1809423 , 1.62059768, 1.17528503, 1.12415308, 2.10443688,
        1.06999147],
       [1.14582379, 0.7214189 , 0.7861035 , 0.42622525, 1.06999147,
        0.66021036]])